# Feature Shuffling For Feature Selection by `Mr. Harshit Dawar!`
* Internal Working of this Method:
    * First of all, it trains the model using the original dataset.
    * Shuffle the Value of any one feature, & then use the dataset as a prediction dataset, & then predicts using the model initially trained & the dataset obtained after shuffling the feature.
    * Record the Performance Drift.
    * Based on the selection metrics, threshold for the Feature Selection is set.
    * Based on the Performance Drift, Features are selected.

In [1]:
# Importing the required Libraries!

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score